<a href="https://colab.research.google.com/github/tyukei/AI/blob/master/report2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#第2回 レポート課題 (12/20 23:55 締切)


# 課題1
### トピック9のnotebook1で示したRNNを用いたIMDBデータのP/N判定のコードを参考に、複数のネットワーク構造を設定して性能の違いを調べよ。また、調べたネットワーク構造の中から比較的性能が高くて学習時間が短いものをひとつ選び、学習データが極端に少ない状況からすべての学習データを用いる状況まで、段階的にデータを増やして学習データの量と識別器の性能の関係を調べよ。 

## 準備

ディープニューラルネットワークでの識別を Keras でコーディングします。Google ColabでGPUを使用するときは、「ランタイム」->「ランタイムのタイプを変更」-> ハードウェアアクセラレータ -> GPU を選ぶ。

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datetime, os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.utils import np_utils
from tensorflow.keras import layers

In [2]:
%load_ext tensorboard

## データセットについて

IMDBデータは映画のレビューに対して、P/N(肯定/否定)のラベルが付いた学習データである。

ここでは、頻度上位10000語を対象とし、データの大きさは先頭の50単語に限定する。

In [11]:
max_features = 10000
maxlen = 50
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)
X_train

array([[2071,   56,   26, ...,   19,  178,   32],
       [8255,    5,    2, ...,   16,  145,   95],
       [ 215,   28,  610, ...,    7,  129,  113],
       ...,
       [   4,   65,  496, ...,    4, 3586,    2],
       [  13,   18,   31, ...,   12,    9,   23],
       [7585,    8, 2197, ...,  204,  131,    9]], dtype=int32)

データは単語ではなく、数字が入っている。これは、次のword_indexのindexに相当し、単語はword_indexのvalueに相当する。

In [4]:
word_index = keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


単語インデックスを単語に戻して、元のデータにデコードする

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])
decoded_review

"grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"

```
grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all
```
は「子供は素晴らしく、絶賛されるべきだ」というポジティブなレビューであった。

出力は0か1である。0がネガティブで1がポジティブな映画のレビューとなっている。

In [6]:
y_train[0]

1

出力は１でポジティブとなっている。

確かにレビューとネガポジ判定は一致していて、データセットはうまく機能していると分かる。


## SimpleRNNを構成して学習させる。

RNNとはRecursive Neural Network(再帰型ニューラルネットワーク)の略である。

Recursiveとあるように中間層で演算結果を再帰的に自身に出力する

具体的には、一時刻前の中間層の出力を次の入力と結合して中間層に入力する。

そのため、前の単語が次の単語に影響をあたえる自然言語処理に適していると言われる。

RNNの学習方法は３ステップである

1. ネットワークの構造の決定
1. fix関数を用いて学習実行
1. テストデータを用いて評価

### ネットワークの構造

```
layers.Embedding(max_features, 128)
```
入力の単語インデックスをone-hotベクトル（インデックスに対応する次元の値が1、それ以外の次元の値は0）とみなしたものを低次元のベクトルに変換するEmbedding（埋め込み）層
```
layers.SimpleRNN(64)
```
 Embeddin層の出力とひとつ前の自身の出力を結合したものに対して重み付き和を求めて活性化関数を適用するSimpleRNN層
 ```
layers.Dense(1, activation='sigmoid')
 ```
 SimpleRNN層の出力の重み付き和を求めて活性化関数を適用し、出力を求めるDense層からなる。KerasではSequentialクラスのインスタンスに対してaddメソッドでこれらの層を積み重ねて識別のためのニューラルネットを構成する。

In [121]:
model1 = keras.Sequential([
    layers.Embedding(max_features, 128),#埋め込み層
    layers.SimpleRNN(64),#outputの次元
    layers.Dense(1, activation='sigmoid')#全結合層
]) 
model1.summary()

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, None, 128)         1280000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense_85 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,292,417
Trainable params: 1,292,417
Non-trainable params: 0
_________________________________________________________________


### 学習
作成したネットワークのcompileメソッドで最適化器・損失関数・評価基準を指定した後、fitメソッドに学習データと学習時のパラメータを与えて学習する。

学習を行う。fit関数を用いる。
主な引数は以下である。 
```
fit(X_train, y_train, epochs=5, batch_size=32,validation_split=0.2)
X_train => 訓練データ
y_train => 教師ラベル
epochs => エポック数
batch_size => バッチサイズ
validation_split => X_trainをさらに分割し、訓練データと検証用に分ける割合
```

fitメソッドの引数validation_splitを指定すると、指定された割合を検証用データとして学習データから分割できる。

1epoch毎に検証用データの判定性能を表示させることができる。

ちなみに、エポック数とは学習回数を表す。
エポック数が少ないと、パラメータが適切に収束する前に学習が終了する。また、あまりにも学習をしすぎると特定のデータのみに強い「過学習」を起こしてしまう。また、学習にも時間がかかる。

バッチサイズは学習データを分割してグループ化(バッチ)したときのデータ数を示す。
バッチサイズが大きいと、局所的解を避けることができ、学習時間を短くすることができる一方で、入力データが平均化され、データの個々の特徴が失われる。一方サイズが小さいと、個々の特徴が反映されるが、時間がかかり、局所解のトラップにかかる可能性が高い。


In [122]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])#訓練プロセス(最適化方、損失関数)
model1.fit(X_train, y_train, epochs=5, batch_size=32,validation_split=0.2)#訓練開始(epochs でエポック,バッチサイズを決める)

Epoch 1/5
625/625 [==============================] - 23s 36ms/step - loss: 0.5204 - acc: 0.7276 - val_loss: 0.5054 - val_acc: 0.7474
Epoch 2/5
625/625 [==============================] - 23s 36ms/step - loss: 0.3125 - acc: 0.8694 - val_loss: 0.4863 - val_acc: 0.7758
Epoch 3/5
625/625 [==============================] - 23s 36ms/step - loss: 0.1468 - acc: 0.9469 - val_loss: 0.6323 - val_acc: 0.7566
Epoch 4/5
625/625 [==============================] - 22s 36ms/step - loss: 0.0587 - acc: 0.9809 - val_loss: 0.8279 - val_acc: 0.7600
Epoch 5/5
625/625 [==============================] - 24s 38ms/step - loss: 0.0394 - acc: 0.9865 - val_loss: 0.9550 - val_acc: 0.7528


### 評価
テストデータを用いて評価を行う。
１に近いほど正答率が高い

In [9]:
test_loss, test_acc = model1.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

782/782 [==============================] - 5s 6ms/step - loss: 0.9539 - acc: 0.7501
Test accuracy: 0.7501


## 双方向LSTM
LSTMはLong Short Term Memory(長・短期記憶)の略である

RNNの勾配消失問題を解消するために考えられたネットワークシステムである。
RNNはデータ数が多いと勾配消失し重みを更新ができなくなる。

RNNが情報をそのまま渡すがLSTMでは中間層を用いて次に渡す。

SimpleRNNでは```layers.SimpleRNN(64)```としていたとろを``` layers.Bidirectional(layers.LSTM(64))```に変更する。引数64はoutputの層の数を示す。

SimpleRNNと同様に、ネットワークの構造決定、学習、評価を行う

In [12]:
model2 = keras.Sequential([
    layers.Embedding(max_features, 128),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 128)         1280000   
                                                                 
 bidirectional_4 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [13]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model2.fit(X_train, y_train, epochs=5, batch_size=32,validation_split=0.2)

Epoch 1/5
625/625 [==============================] - 55s 83ms/step - loss: 0.4605 - acc: 0.7723 - val_loss: 0.4155 - val_acc: 0.8052
Epoch 2/5
625/625 [==============================] - 48s 77ms/step - loss: 0.2896 - acc: 0.8783 - val_loss: 0.4363 - val_acc: 0.7988
Epoch 3/5
625/625 [==============================] - 49s 78ms/step - loss: 0.1700 - acc: 0.9340 - val_loss: 0.5522 - val_acc: 0.7888
Epoch 4/5
625/625 [==============================] - 47s 75ms/step - loss: 0.0841 - acc: 0.9710 - val_loss: 0.7701 - val_acc: 0.7794
Epoch 5/5
625/625 [==============================] - 45s 72ms/step - loss: 0.0422 - acc: 0.9862 - val_loss: 0.8574 - val_acc: 0.7796


In [14]:
test_loss, test_acc = model2.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

782/782 [==============================] - 13s 17ms/step - loss: 0.8401 - acc: 0.7876
Test accuracy: 0.7876


## 3階層ネットワーク
1番目の隠れユニット層を128
2番目の隠れユニット層を64
それぞれ活性化関数はシグモイドを使用する。


In [82]:
model3 = keras.Sequential([
layers.Dense(128, activation='sigmoid', input_shape=(50,)),
layers.Dense(64, activation='sigmoid'),
layers.Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 128)               6528      
                                                                 
 dense_51 (Dense)            (None, 64)                8256      
                                                                 
 dense_52 (Dense)            (None, 1)                 65        
                                                                 
Total params: 14,849
Trainable params: 14,849
Non-trainable params: 0
_________________________________________________________________


In [83]:
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model3.fit(X_train, y_train, epochs=5, batch_size=32,validation_split=0.2)

Epoch 1/5
625/625 [==============================] - 2s 3ms/step - loss: 0.6972 - acc: 0.5028 - val_loss: 0.6957 - val_acc: 0.5052
Epoch 2/5
625/625 [==============================] - 2s 3ms/step - loss: 0.6949 - acc: 0.5141 - val_loss: 0.6989 - val_acc: 0.4958
Epoch 3/5
625/625 [==============================] - 2s 3ms/step - loss: 0.6934 - acc: 0.5179 - val_loss: 0.6940 - val_acc: 0.5020
Epoch 4/5
625/625 [==============================] - 2s 3ms/step - loss: 0.6936 - acc: 0.5182 - val_loss: 0.6938 - val_acc: 0.5008
Epoch 5/5
625/625 [==============================] - 2s 3ms/step - loss: 0.6919 - acc: 0.5228 - val_loss: 0.6962 - val_acc: 0.5086


In [84]:
test_loss, test_acc = model3.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

782/782 [==============================] - 2s 2ms/step - loss: 0.6964 - acc: 0.5060
Test accuracy: 0.506


## 結果
結果をまとめると以下のようになった。

| ネットワーク名 | 処理時間(ms/step)| スコア(accuracy) |
| ---: | ---: | ---: |
|SimpleRNN|35.4 | 0.750|
|LSTM |9.2|0.782|
|3層ニューラルネットワーク|3 |0.506|

結果処理速度が短いのはLSTM, RNN, 3NNの順となった

スコアの良いのはLSTM, RNN, 3NNの順となった。

次にＬＳＴＭのデータ数を変更してみる



## 学習データが極端に少ない状況からすべての学習データを用いる状況まで、段階的にデータを増やして学習データの量と識別器の性能をしらべる

まず、データの大きさをかえてみる。
データの大きさを１、１０、５０、１００で実験を行う。

In [4]:
max_features = 10000
maxlen = 1
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [6]:
model4 = keras.Sequential([
    layers.Embedding(max_features, 128),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(1, activation='sigmoid')
])
model4.summary()

model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model4.fit(X_train, y_train, epochs=2, batch_size=32,validation_split=0.2)

test_loss, test_acc = model4.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
625/625 [==============================] - 16s 21ms/step - loss: 0.6644 - acc: 0.5790 - val_loss: 0.6558 - val_acc: 0.5834
Epoch 2/2
782/782 [==============================] - 2s 2ms/step - loss: 0.6684 - acc: 0.5864
Test accuracy: 

In [7]:
max_features = 10000
maxlen = 10
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [8]:
model5 = keras.Sequential([
    layers.Embedding(max_features, 128),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(1, activation='sigmoid')
])
model5.summary()

model5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model5.fit(X_train, y_train, epochs=2, batch_size=32,validation_split=0.2)

test_loss, test_acc = model5.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
625/625 [==============================] - 25s 30ms/step - loss: 0.5674 - acc: 0.6912 - val_loss: 0.5329 - val_acc: 0.7228
Epoch 2/2
782/782 [==============================] - 4s 5ms/step - loss: 0.5440 - acc: 0.7204
Test accuracy: 

In [9]:
max_features = 10000
maxlen = 100
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [10]:
model6 = keras.Sequential([
    layers.Embedding(max_features, 128),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(1, activation='sigmoid')
])
model6.summary()

model6.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model6.fit(X_train, y_train, epochs=2, batch_size=32,validation_split=0.2)

test_loss, test_acc = model6.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         1280000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
625/625 [==============================] - 80s 123ms/step - loss: 0.4152 - acc: 0.8078 - val_loss: 0.3435 - val_acc: 0.8464
Epoch 2/2
782/782 [==============================] - 23s 29ms/step - loss: 0.4211 - acc: 0.8395
Test accurac

In [ ]:
max_features = 10000
maxlen = 10
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

## 結果
結果は以下のようになった

|データ数|処理時間ms/step|スコアaccuracy|
|---:|---:|---:|
|1|15|0.5864|
|10|29.5|0.7204|
|50|9.2|0.7821|
|100|124|0.8395|

データの数を増やすとスコアは上がった。一方で、処理速度は大きくなった。
データ数が増えると、レビューの文字数が増えるため、より多くの特徴量を抽出することができ、学習性能が上がったと考えられる。

# 課題2
### トピック9のnotebook2のコードを参考に、事前学習モデルを用いる設定で、学習データが極端に少ない状況からすべての学習データを用いる状況まで、段階的にデータを増やして学習データの量と識別器の性能の関係を調べよ。

まず、ライブラリや設定をおこなう

In [15]:

import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import Dense

# Load compressed models from tensorflow_hub
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.9.2
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


データをインポートする

In [16]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

X_train, y_train = tfds.as_numpy(train_data)
X_test, y_test = tfds.as_numpy(test_data)
     

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0L0564/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0L0564/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0L0564/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


事前学習モデルの設定する

In [17]:
embed = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embed, input_shape=[], dtype=tf.string, trainable=True)
     

事前学習モデルを用いてネットワーク構造を決める

In [18]:
model7 = keras.Sequential([
    hub_layer,
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
]) 
model7.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense_5 (Dense)             (None, 16)                816       
                                                                 
 dense_6 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


学習を行う

In [23]:
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model7.fit(X_train, y_train, epochs=3, batch_size=512, validation_split=0.2)

Epoch 1/3
40/40 [==============================] - 27s 673ms/step - loss: 0.0260 - acc: 0.9977 - val_loss: 0.3690 - val_acc: 0.8720
Epoch 2/3
40/40 [==============================] - 25s 616ms/step - loss: 0.0144 - acc: 0.9993 - val_loss: 0.4014 - val_acc: 0.8712
Epoch 3/3
40/40 [==============================] - 24s 612ms/step - loss: 0.0089 - acc: 0.9997 - val_loss: 0.4294 - val_acc: 0.8700


最後に評価を行う

In [24]:
score = model7.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]:.4}')
print(f'Test accuracy: {score[1]:.4}')

Test loss: 0.4826
Test accuracy: 0.8517


どうようにデータ数を10、50、100、500、1000で実験を行う。

In [26]:
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model7.fit(X_train[:10], y_train[:10], epochs=3, batch_size=10, validation_split=0.2)

Epoch 1/3
1/1 [==============================] - 2s 2s/step - loss: 0.0079 - acc: 1.0000 - val_loss: 0.0071 - val_acc: 1.0000
Epoch 2/3
1/1 [==============================] - 1s 872ms/step - loss: 0.0038 - acc: 1.0000 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 3/3
1/1 [==============================] - 1s 672ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.0065 - val_acc: 1.0000


In [27]:
score = model7.evaluate(X_test[:2], y_test[:2], verbose=0)
print(f'Test loss: {score[0]:.4}')
print(f'Test accuracy: {score[1]:.4}')

Test loss: 1.44
Test accuracy: 0.5


In [28]:
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model7.fit(X_train[:50], y_train[:50], epochs=3, batch_size=10, validation_split=0.2)

Epoch 1/3
4/4 [==============================] - 4s 1s/step - loss: 0.0106 - acc: 1.0000 - val_loss: 0.0044 - val_acc: 1.0000
Epoch 2/3
4/4 [==============================] - 2s 623ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 3/3
4/4 [==============================] - 2s 622ms/step - loss: 5.1913e-04 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000


In [29]:
score = model7.evaluate(X_test[:10], y_test[:10], verbose=0)
print(f'Test loss: {score[0]:.4}')
print(f'Test accuracy: {score[1]:.4}')

Test loss: 0.9947
Test accuracy: 0.7


In [30]:
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model7.fit(X_train[:100], y_train[:100], epochs=3, batch_size=10, validation_split=0.2)

Epoch 1/3
8/8 [==============================] - ETA: 0s - loss: 0.0080 - acc: 1.0000

8/8 [==============================] - 6s 679ms/step - loss: 0.0080 - acc: 1.0000 - val_loss: 0.0255 - val_acc: 1.0000
Epoch 2/3
8/8 [==============================] - 5s 623ms/step - loss: 8.8917e-04 - acc: 1.0000 - val_loss: 0.0187 - val_acc: 1.0000
Epoch 3/3
8/8 [==============================] - 5s 653ms/step - loss: 3.8195e-04 - acc: 1.0000 - val_loss: 0.0181 - val_acc: 1.0000


In [31]:
score = model7.evaluate(X_test[:20], y_test[:20], verbose=0)
print(f'Test loss: {score[0]:.4}')
print(f'Test accuracy: {score[1]:.4}')

Test loss: 0.5666
Test accuracy: 0.85


In [32]:
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model7.fit(X_train[:500], y_train[:500], epochs=3, batch_size=10, validation_split=0.2)

Epoch 1/3
40/40 [==============================] - ETA: 0s - loss: 0.0078 - acc: 1.0000

40/40 [==============================] - 28s 674ms/step - loss: 0.0078 - acc: 1.0000 - val_loss: 0.0078 - val_acc: 1.0000
Epoch 2/3
40/40 [==============================] - 24s 608ms/step - loss: 9.3857e-04 - acc: 1.0000 - val_loss: 0.0103 - val_acc: 1.0000
Epoch 3/3
40/40 [==============================] - 24s 603ms/step - loss: 2.3625e-04 - acc: 1.0000 - val_loss: 0.0100 - val_acc: 1.0000


In [33]:
score = model7.evaluate(X_test[:50], y_test[:50], verbose=0)
print(f'Test loss: {score[0]:.4}')
print(f'Test accuracy: {score[1]:.4}')

Test loss: 0.9547
Test accuracy: 0.82


In [34]:
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model7.fit(X_train[:1000], y_train[:1000], epochs=3, batch_size=10, validation_split=0.2)

Epoch 1/3
80/80 [==============================] - 52s 642ms/step - loss: 0.0061 - acc: 0.9987 - val_loss: 0.0167 - val_acc: 0.9950
Epoch 2/3
80/80 [==============================] - 60s 755ms/step - loss: 2.4907e-04 - acc: 1.0000 - val_loss: 0.0059 - val_acc: 1.0000
Epoch 3/3
80/80 [==============================] - 52s 652ms/step - loss: 1.2063e-04 - acc: 1.0000 - val_loss: 0.0060 - val_acc: 1.0000


In [35]:
score = model7.evaluate(X_test[:200], y_test[:200], verbose=0)
print(f'Test loss: {score[0]:.4}')
print(f'Test accuracy: {score[1]:.4}')

Test loss: 0.7446
Test accuracy: 0.84


## 結果
結果は以下のようになった。

|データ数|処理時間ms/step|スコアaccuracy|損出loss|
|---:|---:|---:|---:|
|10|1,181|0.50|1.44|
|50|748|0.70|0.9947|
|100|652|0.85|0.5666|
|500|628|0.82|0.9547
|1000|683|0.84|0.7446|
|2500|634| 0.85|0.4826|


最もスコアが悪いのはデータ数が最も少ない10の時であった。
学習量が少なく、パラメータが適切に収束する前に学習が終了したためだと考えられる。

一方で最もスコアが高かったのはデータ数が100のときと2500のときであった。
データ数が100～2500の間ではスコアは0.3の範囲であった。
データ数が100から100にかけては若干落ちるところがあった。
100のとき偶然最適解を見つけられたが、他のデータ数ではデータ数が足りず、局所解のトラップに入ってスコアがあまり伸びなかったと考えられる。

損出はスコアに反比例して、スコアがの値がいいと(accuracyが低いと)損失は大きくなる。損出は正解ラベルと出力値の差であり、差が小さいほど正答率は高まりスコアが伸びることからもわかる。

データ数が10から500にかけて増えるほど、処理時間は小さくなった。
500から1000にかけては一度処理時間は増えたが、2500では再び減少をした。

